In [4]:
# merf psd 0.7742113806103438
# merf after reref to avg mastoids 0.7436573207110183
#psd - pid - all - KNeighborsClassifier          - 0.750

# TODO: proper regression model, after filter(elo >= 1000 & elo <= 1750) 
#                                      standardize elo within participant
#                                
#TODO: ADD OTHER REGRESSORS TO MERF. 
#TODO: ensure that we have balanced data per participant in training/testing. 
#TODO: see how hold-one-out participant training works; see if adding some training data per person works. 
 

# STUFF TRIED BUT NOT NECESSARY
# not doing for stats anymore, so keep em! 
# drop overall bottom and top 10% of elo
# df_mean['elo'] = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile').fit_transform(df_mean['elo'].to_numpy().reshape(-1, 1)).astype(int)
# df_mean = df_mean[(df_mean['elo'] != 0) & (df_mean['elo'] != 9)] 

# # min-max scale all the elos - since we're discretizing, this isn't necessary
# elos = df_mean['elo'] 
# df_mean['elo'] = (elos - elos.min()) / (elos.max() - elos.min())
# df_mean['elo'] *= 100
# df_mean['elo'] = df_mean['elo'].astype(int)

# AVERAGE_PROBES = False

#     if AVERAGE_PROBES:
#         for wave in ['alpha', 'beta', 'delta', 'theta']:
#             df_mean[f"{wave}_avg"] = (df_mean[f'{wave}.probe-0'] + df_mean[f'{wave}.probe-1'] + 
#                                       df_mean[f'{wave}.probe-2'] + df_mean[f'{wave}.probe-3']) / 4
#             df_mean[f"{wave}_sd"]  = (df_sd[f'{wave}.probe-0'] + df_sd[f'{wave}.probe-1'] +
#                                       df_sd[f'{wave}.probe-2'] + df_sd[f'{wave}.probe-3']) / 4
            
#         df_mean = df_mean.drop(columns = [x for x in df.keys() if 'probe' in x])


import pandas as pd
import os
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, \
                             AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.mixture import GaussianMixture
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report, r2_score
from merf import MERF
from library import *

MERF_MAX_ITER = 20

np.random.seed(RANDOM_STATE)

NORMALIZE = False

Y_STYLE = 'elo'
dtype   = "psd"

df = pd.read_csv(f"{DBFNAME.split('.csv')[0]}_psd_{FILTNAME}", dtype = {'pid': str})
df = df[df['pid'].notna()]
df = df[df['pid'] != "92511e53"]    # this guy is WAY better than everyone else; remove his data.

PIDS = psd_df['pid'].unique()

def extract_features(df):
    print(df)

    df_mean = df.groupby(['pid', 'block_id']).mean().reset_index()
    df_sd = df.groupby(['pid', 'block_id']).std().reset_index()
 
    #print(df_mean)        

    # use top and bottom 1/3 of elo for classification
    df_mean['elo'] = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile').fit_transform(df_mean['elo'].to_numpy().reshape(-1, 1)).astype(int)
    print(df_mean['elo'].describe())
    #df_mean = df_mean[(df_mean['elo'] != 1)]
    # 2s -> 1s ==> turn to binary
    #    df_mean['elo'] = df_mean['elo'].replace(2, 1) 
  
    # split into 7 bins, choose bottom 2 and top 4 => 4/7 => yields 80% accuracy
    # df_mean['elo'] = KBinsDiscretizer(n_bins=7, encode='ordinal', strategy='quantile').fit_transform(df_mean['elo'].to_numpy().reshape(-1, 1)).astype(int)
    # df_mean = df_mean[(df_mean['elo'] <= 1) | (df_mean['elo'] >= 5)]
    # df_mean['elo'] = df_mean['elo'].replace(1, 0)
    # df_mean['elo'] = df_mean['elo'].replace(5, 1)
    # df_mean['elo'] = df_mean['elo'].replace(6, 1)

    # use if we want to add standard deviation
    # for key in [k for k in df_sd.keys() if 'probe' in k]:
    #     df_mean[key + '.sd'] = df_sd[key]
    
    df = df_mean

    p_dfs = {}
    for pid in df['pid'].unique():
        p_df = df[df['pid'] == pid].copy().reset_index(drop=True)
           
        if Y_STYLE == 'elo': p_df['y'] = p_df['elo']
        else:                p_df['y'] = p_df['solved']

        p_dfs[pid] = p_df #.sample(n=35, replace=False, random_state=RANDOM_STATE) 
    
        print(f"0 - {sum(p_dfs[pid]['y'] == 0)} - 1 - {sum(p_dfs[pid]['y'] == 1)}")
    return p_dfs
    
ALL_DFS = extract_features(psd_df)

all_X = []
all_y = []
all_P = []
for i, p in enumerate(PIDS):

    
    print(f"number of puzzles for {p}: {len(ALL_DFS[p])}")
    
    p_subset_df = ALL_DFS[p]
    y = p_subset_df['y'].to_numpy()
    X = p_subset_df.drop(columns=['y', 'pid', 'block_id', 'timestamp', 'elo', 'solved']).to_numpy()
    all_X.extend(X)
    all_y.extend(y)
    all_P.extend(pd.Series([i] * len(X)))

    #runCV(X, y, p, dtype)
print('---------------------------------')
print('---------------------------------')

runCV(all_X, all_y, 'all', dtype)
# run_merf(all_X, all_y, all_P, dtype)    

print('---------------------------------')
print('---------------------------------')


        delta.probe-0  delta.probe-1  delta.probe-2  delta.probe-3  \
0            1.622315       2.769667       2.723968       1.741097   
1            1.954375       2.823065       3.079325       1.859442   
2            2.286995       2.925057       3.408145       2.124228   
3            2.493395       3.043359       3.613963       2.430442   
4            2.631627       3.258771       3.762355       2.656391   
...               ...            ...            ...            ...   
139334       2.469729       2.367149       2.539648       2.439549   
139335       2.263767       2.162679       2.329796       2.230391   
139336       1.941438       1.798132       2.003060       1.859764   
139337       1.554543       1.289372       1.584606       1.305354   
139338       1.129414       0.690161       1.245645       0.766861   

        theta.probe-0  theta.probe-1  theta.probe-2  theta.probe-3  \
0            1.827639       2.313439       1.727301       1.537712   
1            1.8090

NameError: name 'runCV' is not defined

In [2]:
def runCV(X, y, pid, datatype):
 
    MODELS = {
        'RandomForestClassifier': {
            'model': RandomForestClassifier(),
            'gridparams': {
                'n_estimators': [100, 200, 300, 500],
                'max_depth': [None, 10, 20, 30],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4],
                'bootstrap': [True, False]
            }
        },
        'GradientBoostingClassifier': {
            'model': GradientBoostingClassifier(),
            'gridparams': {
                'learning_rate': [0.1, 0.05, 0.01],
                'n_estimators': [100, 200, 500],
                'max_depth': [3, 5, 10],
                'min_samples_split': [2, 5, 10]
            }
        },
        'KNeighborsClassifier': {
            'model': KNeighborsClassifier(),
            'gridparams': {
                'n_neighbors': range(1, 30),
                'weights': ['uniform', 'distance'],
                'metric': ['euclidean', 'manhattan', 'minkowski']
            }
        },
        'AdaBoostClassifier': {
            'model': AdaBoostClassifier(),
            'gridparams': {
                'n_estimators': [50, 100, 200],
                'learning_rate': [0.01, 0.1, 1.0]
            }
        },
        'SVC': {
            'model': SVC(),
            'gridparams': {
                'C': [0.1, 1, 10, 100],
                'gamma': ['scale', 'auto', 0.1, 1, 10],
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
            }
        }
    }

    cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE) # 5-Fold CV
    reports = { model: [] for model in MODELS }
    for modelName in MODELS:
        model = MODELS[modelName]['model']
        params = MODELS[modelName]['gridparams']
        grid_search = GridSearchCV(estimator=model, param_grid=params, 
                                   cv=cv, scoring='f1_macro', n_jobs=-1)
        
        grid_search.fit(X, y)
        
        print(modelName)
        print("Best parameters:", grid_search.best_params_)
        print("Best cross-validation score:", grid_search.best_score_)
        reports[model] = [grid_search.best_score_]

    avgs = {model:np.mean(reports[model]) for model in reports}
    max_name_length = max(len(model.__class__.__name__) for model in avgs)

    # print best modelss sorted by avg f1 score
    avgs = {k: v for k, v in sorted(avgs.items(), key=lambda item: item[1], reverse=True)}
    for model, avg in avgs.items():
        print(f"{datatype} - pid - {pid} - {model.__class__.__name__:<{max_name_length}} - {avg:.3f}")
    
    print('---------------------------------')

In [3]:
def run_merf(X, y, P, datatype):  
    
    MERF_REGRESSOR = RandomForestRegressor(random_state=RANDOM_STATE)

    X = np.array(X)
    y = np.array(y)
    P = pd.Series(P)  

    n_splits = 3

    kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    
    indices = list(range(len(X)))

    reports = []
    for i, (train_indices, test_indices) in enumerate(kf.split(indices)):
        print(f"MERF i: {i}" )
        train_indices = train_indices.tolist()
        test_indices = test_indices.tolist()
        
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        P_train, P_test = P[train_indices], P[test_indices]
        
        Z_train = np.ones((len(X_train), 1))
        Z_test = np.ones((len(X_test), 1))
                    
        mrf = MERF(MERF_REGRESSOR, max_iterations=MERF_MAX_ITER)        
        mrf.fit(X_train, Z_train, P_train, y_train)
        
        preds = mrf.predict(X_test, Z_test, P_test)
        
        preds = [pred > 0.5 for pred in preds] # duh this isn't super great 
        print(y_test)
        print(preds)
        
        report = classification_report(y_test, preds, output_dict=True)        
        reports.append(report['macro avg']['f1-score'])
    
    print(f"{datatype} - MERF macro f1-score average on test set for {n_splits}-fold cross-validation: {np.mean(reports)}")